In [1]:
import pandas as pd

crashes = pd.read_csv('../data/crashes_crashes.csv')

people = pd.read_csv('../data/crashes_people.csv')

vehicles = pd.read_csv('../data/crashes_vehicles.csv')




/Users/nechamaborisute/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,23,24,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/nechamaborisute/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (18,20,39,40,41,43,47,48,49,52,54,57,58,60,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
crashes_prime_cause = crashes[(crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE') & (crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'NOT APPLICABLE')]

crashes_prime_cause_filled = crashes_prime_cause.fillna({'INTERSECTION_RELATED_I': 'N', 'NOT_RIGHT_OF_WAY_I': 
                                                         'N', 'HIT_AND_RUN_I':'N'})

crashes_prime_cause_filleddrop = crashes_prime_cause_filled.dropna(subset=['LATITUDE', 'LONGITUDE', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'MOST_SEVERE_INJURY'])


from sklearn.cluster import KMeans

n_clusters = 30  # Number of clusters to create
X = crashes_prime_cause_filleddrop[['LONGITUDE', 'LATITUDE']]

# Create a K-Means clustering model
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

# Add cluster labels to your data
cluster_labels = kmeans.labels_
crashes_prime_cause_filleddrop['GEO_KMEANS_Cluster'] = cluster_labels





crashes_prime_cause_filleddrop = crashes_prime_cause_filleddrop.drop(columns = ['CRASH_DATE_EST_I', 'DEVICE_CONDITION', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED',
                                                          'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME','BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 
                                                          'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 
                                                          'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION', 
                                                          'INJURIES_UNKNOWN', 'LOCATION', 'LANE_CNT', 'CRASH_DATE', 
                                                          'INJURIES_REPORTED_NOT_EVIDENT', 'TRAFFIC_CONTROL_DEVICE', 'INJURIES_TOTAL', 
                                                          'INJURIES_FATAL'])
crashes_prime_cause_filleddrop.info()


<ipython-input-2-5cf993a9d78a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_prime_cause_filleddrop['GEO_KMEANS_Cluster'] = cluster_labels


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442382 entries, 2 to 798905
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CRASH_RECORD_ID          442382 non-null  object 
 1   POSTED_SPEED_LIMIT       442382 non-null  int64  
 2   WEATHER_CONDITION        442382 non-null  object 
 3   LIGHTING_CONDITION       442382 non-null  object 
 4   FIRST_CRASH_TYPE         442382 non-null  object 
 5   TRAFFICWAY_TYPE          442382 non-null  object 
 6   ALIGNMENT                442382 non-null  object 
 7   ROADWAY_SURFACE_COND     442382 non-null  object 
 8   ROAD_DEFECT              442382 non-null  object 
 9   CRASH_TYPE               442382 non-null  object 
 10  INTERSECTION_RELATED_I   442382 non-null  object 
 11  NOT_RIGHT_OF_WAY_I       442382 non-null  object 
 12  HIT_AND_RUN_I            442382 non-null  object 
 13  DAMAGE                   442382 non-null  object 
 14  PRIM

In [3]:
#Transforming to numerical data

crashes_prime_cause_filleddrop_dummy = pd.get_dummies(crashes_prime_cause_filleddrop, columns = ['WEATHER_CONDITION', 'LIGHTING_CONDITION',
                                                                                                 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 
                                                                                                 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 
                                                                                                 'SEC_CONTRIBUTORY_CAUSE', 'ROAD_DEFECT',
                                                                                                 'MOST_SEVERE_INJURY'], drop_first=True, 
                                                                                                  dtype=int)
                                                                                                 




#binary/tertiary
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['CRASH_TYPE','INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE']:
    crashes_prime_cause_filleddrop_dummy[col] = le.fit_transform(crashes_prime_cause_filleddrop_dummy[col])


crashes_cleaned = crashes_prime_cause_filleddrop_dummy


crashes_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 442382 entries, 2 to 798905
Columns: 127 entries, CRASH_RECORD_ID to MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT
dtypes: float64(2), int32(1), int64(122), object(2)
memory usage: 430.3+ MB


In [4]:
crashes_cleaned['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

FAILING TO YIELD RIGHT-OF-WAY                                                       87034
FOLLOWING TOO CLOSELY                                                               77351
IMPROPER OVERTAKING/PASSING                                                         39012
FAILING TO REDUCE SPEED TO AVOID CRASH                                              33568
IMPROPER BACKING                                                                    31286
IMPROPER LANE USAGE                                                                 28331
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                 26380
IMPROPER TURNING/NO SIGNAL                                                          26339
DISREGARDING TRAFFIC SIGNALS                                                        15440
WEATHER                                                                             11870
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER    10097
DISREGARDI

In [5]:
crashes_cleaned.to_csv('../data/crashes_cleaned.csv', index=True)